#大数据分析 Lesson1

## PART 1  大规模数据的线性回归(Sketching方法)


普通的线性回归： $y=AX+b+\varepsilon=A'X+\varepsilon$, dimA=(m,n), dimA'=(m,n+1),即把b当成A的新列看待。之后把A'写作A. $\varepsilon$是噪声

那么LR的目标是$min\varepsilon \equiv min||AX-y||_{2}$。当A的规模极大时，做普通的LR是非常耗时的。

回顾：如何做线性回归？（Sklearn走开！）

解析视角下：$y=Ax \Rightarrow x=A^{-1}y $, 如果A不是满秩矩阵则求它的伪逆。

迭代视角下： 若不用矩阵表达，则目标函数为$min\frac{1}{2}\sum_{i=1}^{m}(X^{T}a^{i}-y^{i})^{2}$,记得数据点是$(a^{i},y^{i})$。对于参数$x_{j}$, 每步迭代增量为$(x^{j}a^{i}-y^{i})a^{i}$。计算上，如果采用Batch方式，则对这批数据全部计算增量后再更新参数；如果是Incremental方式，则每一个数据点更新一次。实际上我们将整个数据集分成若干个batch，每个batch内运行Batch方式。

Recall: 目标函数（即最小二乘法）的概率解释：满足高斯分布的噪声$\varepsilon$的极大似然估计。

矩阵视角下的计算法：



当数据A的规模极大时，我们采用sketching的方法构造矩阵S,使得目标变成$x = argmin_{x}||(SA)X-(Sy)||_{2}$

**复杂度**：减少时间复杂度，poly内求解。

**精确度**：既然提升了效率，就要牺牲精确度。我们构造的S，有0.99的置信度，使$||Sy||_{2}=(1 \pm \varepsilon)||y||_{2}$。

构造S的方法：sparse embedding matrix。S的行数$r=O(\frac{n}{\varepsilon^{2}})$，列数=A行数=m，dimS=(r,m)。

复杂度分析：矩阵乘法SA为O(nnz(A))，进行subspace embedding之后再去LR的复杂度为$O(\frac{n}{\varepsilon})$。总复杂度为O(nnz(A))+$O(\frac{n}{\varepsilon})$。

In [0]:
# sparse embedding matrix S
import numpy as np

epsilon = 0.99
m = 10000
n = 10
nonzero = [-1,1]
row = int(n/np.power(epsilon,2))
col = m
s = np.zeros((row,col))
for i in range(col):
  entry = int(np.random.randint(0,row,1))
  j = int(np.random.randint(0,2,1))
  s[entry][i] = nonzero[j]

s


array([[ 0., -1.,  0., ..., -1.,  0., -1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

##PART2: SVD分解

数据降维的方式，一种是经典的PCA，通过特征分解数据的协方差阵寻找最大的d个主成分作为新空间的基，最小化重建误差，同时使这d个方向上的方差是前d个最大的并且相互不重叠信息。而SVD通过k秩近似法，寻找组成低维空间能使最小化重建误差的新矩阵的小矩阵作为大矩阵的近似表示。PCA从名字上就很直观，找到矩阵的主成分，也就意味这从一出生这就是个降维的方法。SVD是一种矩阵分解方法，相当于因式分解，他的目的纯粹就是将一个矩阵拆分成多个矩阵相乘的形式。并用这些小矩阵近似表示大矩阵。
[PCA和SVD的区别与联系](https://www.cnblogs.com/bjwu/p/9280492.html)

---


SVD：$dimA=(m,n)。 dimU=(m,m), dimV=(n,n), dim\Sigma=(m,n)$, U、V都是正交阵。$\Sigma$是对角阵（如果不是方阵，则其他余下的部分都是0）则有：

$A=U\Sigma V^{T}$（分解的过程在上面链接中）

注意到：
$AA^{T}=U\Sigma^{2}U^{T}, A^{T}A=V\Sigma^{2}V^{T}$,因此求奇异值矩阵$\Sigma$时对$AA^{T}$阵做特征分解即可。$AA^{T}$的特征值为$\lambda_{i}$时，对应的奇异值为$\sigma_{i}=\lambda_{i}^{1/2}$。求奇异矩阵U V时，只需要分别对$AA^{T} 和A^{T}A$做正交对角化，拿出它们的特征向量组成矩阵即可。

对原矩阵降维：按顺序寻找前k个奇异值，即topK, $B=U^{*}\Sigma^{*}V^{*T}$, $dimU^{*}$=(m,k), $dim\Sigma^{*}=(k,k)$, $dimV^{*}$=(k,n)。使得$||B-A||_{F}$最小。因此大矩阵A可以用三个小矩阵U* $\Sigma$* V* 描述。




# PART3: MapReduce with Perception

##Recall Perception:

解决线性二分类问题,y=+1/-1。多分类问题，train多个二分类感知机，找y'最高的对应的类别。

阈值为0的感知机：训练参数w, y'=wx

阈值为$\theta$的感知机，w->[w,$\theta$], x->[x,-1]，形式上还是y'=wx

感知机的决策面是超平面wx=0，尽力将两类凸集分离。只要得到了w的值，就能画出与w垂直的超平面。

训练过程（online/batch/mini-batch）:


---


Loop until converge:
1.   Compute y'=wx
2.  if yy'<=0:  w <- w+$\eta$yx



---



##MapReduce with Perception

把数据部署到分布式系统上计算，每个机器得到一个data chunk。每个map机器都同步相同的权值w，之后reduce机器更新w后广播给map机器。

---
map:


对chunk里的每一个数据(x,y)，计算y'=wx。若y'y<0，则生成若干个key-value pairs (i, $\eta yx_{i}$), i是x非零分量的脚标。对于分量为0的$x_{i}$，它对权重更新的贡献也是0，因此没有必要计算。

reduce:


汇总所有的(i, $\eta yx_{i}$) 对，对w做更新：
$w_{i}=w_{i}+$\eta yx_{i}$

---







